# Data Preparation


In [1]:
#importing libraries:

import numpy as np
from pandas import Series,DataFrame
import pandas as pd
import datetime as dt
import janitor
import matplotlib



Importing all data sources:

In [2]:
#Contracts:
contracts =  pd.read_excel('Contracts June Snapshot -Active and ND.xlsx')
contracts =  pd.DataFrame(contracts)

# Creating a coulumn for contract_line_number:
contracts['contract_line_number'] = contracts.ContractNumber.astype(str) + "-" + contracts.ContractItemNumber.astype(str)

# Selecting only VBR and Support Services records and restoring contracts to those:
contracts_filtered = contracts[contracts['Portfolio'].isin(["VBR", "Support Services"])]
contracts = pd.DataFrame(contracts_filtered)


In [3]:
#ITSM:
itsm = pd.read_excel('itsm.xlsx').clean_names()

#Manfacturer_mapping:
manu_mapping =  pd.read_excel('Manufacturer_Std_Mapping V1_new.xlsx').clean_names()


In [4]:
#PO:
po = pd.read_excel("PO_1.xlsx")
po['SalesDocumentItemNumber'] = po['SalesDocumentItemNumber'].astype(np.int64)

# Creating a coulumn for sales_document_line_number:
po['sales_document_line_number'] = po.SalesDocumentNumber.astype(str) +'-'+ po.SalesDocumentItemNumber.astype(str)


# Calculating missing NetValueUSD for non-zero ACVBUDGETUSD:

In [5]:
mask1 = contracts["ACVUSDBudget"] != 0
mask2 = contracts["NetValueUSD"] == 0

contracts_filter = contracts[mask1 & mask2]

contracts_filter["date_diff_days"] = pd.to_datetime(contracts_filter["End Date"]) - pd.to_datetime(contracts_filter["Start Date"])
contracts_filter["ACV_per_day"] = contracts["ACVUSDBudget"]/365
contracts_filter["NetValueUSD"] = contracts_filter["ACV_per_day"] * (contracts_filter["date_diff_days"]).dt.days
contracts_filter = contracts_filter[['contract_line_number','NetValueUSD']]

#merging back to contracts:
contracts_merged_final = contracts.merge(contracts_filter, how="left", on = "contract_line_number")
contracts_merged_final["NetValueUSD_y"].fillna(contracts_merged_final["NetValueUSD_x"], inplace= True)

#Taking only non-zero (ACVBudgetUSD and NetValueUSD_y) observations:
mask1 = contracts_merged_final["ACVUSDBudget"] != 0
mask2 = contracts_merged_final["NetValueUSD_y"] != 0

contracts_merged_final = contracts_merged_final[mask1 & mask2]

#Removing NetValueUSD_x as we have adjusted net value for non zero ACVBudgetUSD in NetValueUSD_y:
contracts_merged_final.drop(['NetValueUSD_x'], axis=1,inplace = True) 

#Renaming NetValueUSD_y to NetValueUSD_x:
contracts_merged_final.rename(columns = {'NetValueUSD_y':'NetValueUSD_x'}, inplace = True)

#Restoring everything to contracts.
contracts = pd.DataFrame(contracts_merged_final) 

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

# Imputing DD-AM,AP,EU, GLOBAL, MEA:

In [6]:
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-GLOBAL", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-AM", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-AP", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-EU", "DD", case = False)
po["Manufacturer"]= po["Manufacturer"].str.replace("DD-MEA", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-GLOBAL", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-AM", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-AP", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-EU", "DD", case = False)
contracts["ManufacturerCode"]= contracts["ManufacturerCode"].str.replace("DD-MEA", "DD", case = False)

In [7]:
contracts["NetValueUSD_x"].sum()

2837275407.590391

# Mapping standard manufactureres names to ITSM from Mapping File:

In [8]:
#renaming itsm_manufacturer to manufacturer in manu_mapping
manu_mapping.rename(columns = {'itsm_manufacturer':'manufacturer'}, inplace = True)

#getting manufacturers standard names in ITSM:
itsm_mapping_merged = itsm.merge(manu_mapping, how="left", on = "manufacturer")

itsm_mapping_merged.standard_mapping.isna().sum()
# 4762 not mapped
pd.notna(itsm_mapping_merged["standard_mapping"]).sum()
# 114311 mapped

#Replacing NAs in Standard Mapping with manufacturer as given in manu_mapping file.
itsm_mapping_merged["standard_mapping"].fillna(itsm_mapping_merged["manufacturer"], inplace= True)

# Resoring to ITSM:
itsm = pd.DataFrame(itsm_mapping_merged).clean_names()

# Merging Contracts, PO and ITSM:

In [9]:
#contract_PO:
contracts_merged_po = contracts.merge(po, how="left", left_on= "contract_line_number", right_on="sales_document_line_number")
contracts_merged_po["merged_po"]= np.where(contracts_merged_po['SalesDocumentNumber'].isna() , 0, 1)

#contracts_po_itsm:
contract_po_itsm_merged = contracts_merged_po.merge(itsm[["contract_line_number","standard_mapping"]], how= "left", on="contract_line_number")
contract_po_itsm_merged["merged_itsm"]= np.where(contract_po_itsm_merged['standard_mapping'].isna() , 0, 1)

#missing itsm manufacturers in contracts_po_itsm:
pd.isna(contract_po_itsm_merged["standard_mapping"]).sum()
# 173812

#here on taking contracts as a single source of all data.
contracts = pd.DataFrame(contract_po_itsm_merged).clean_names()

# Vendor Standardisation:

In [10]:
contracts_manufac_filter = contracts[["contract_line_number","manufacturer","manufacturercode","vendor","standard_mapping"]]
vendor_mapper =pd.DataFrame(manu_mapping)

vendor_mapper.rename(columns = {'manufacturer':'vendor'}, inplace = True)
vendor_mapper.rename(columns = {'standard_mapping':'standard_mapping_vendor'}, inplace = True)
contracts_manufac_filter_1 = contracts_manufac_filter[["contract_line_number","vendor"]].merge(vendor_mapper, how="left", on = "vendor")

#Replacing NA in standardised_vendor with existing vendor missing in vendor_mapper:
contracts_manufac_filter_1["standard_mapping_vendor"].fillna(contracts_manufac_filter_1["vendor"], inplace= True)

contracts_manufac_filter_1= pd.DataFrame(contracts_manufac_filter_1.drop(contracts_manufac_filter_1.index[390786:390818]))

#Arranging Contract_line_number and Vendor for cbind:
contracts = contracts.sort_values(by=["contract_line_number","vendor"])
contracts_manufac_filter_1 = contracts_manufac_filter_1.sort_values(by=["contract_line_number","vendor"])

#cbinding standardised vendor to contracts:
contracts = contracts.drop(columns=["vendor"])
contracts_manufac_filter_1 = contracts_manufac_filter_1.drop(columns=["vendor"])
contracts = pd.concat([contracts, contracts_manufac_filter_1], axis=1)

#removing extra "Contract_Line_Number col. "
contracts=contracts.drop(columns=contracts.columns[85])
contracts['contract_line_number'] = contracts.contractnumber.astype(str) + "-" + contracts.contractitemnumber.astype(str)

# renaming standard_mapping_vendor to vendor
contracts.rename(columns = {'standard_mapping_vendor':'vendor'}, inplace = True)

In [11]:
contracts.columns

Index(['contractnumber', 'contractitemnumber', 'precedingdocumentnumber',
       'start_date', 'end_date', 'soldtopartyname', 'soldtopartycode_x',
       'clientdomaincode_x', 'clientdomaindesc',
       'adjusted_client_domain_description', 'country', 'region',
       'manufacturerpartnumber_x', 'status', 'contractstatusflag',
       'account_assignment', 'portfolio', 'plan', 'monitoringflag', 'industry',
       'bu', 'cancellationreasondescription', 'cancellation_updated_date',
       'contract_term', 'documentcurrencycode', 'acv', 'snapshot',
       'acvusdbudget', 'fromdirect', 'confirmation_no_', 'baselineflag',
       'renewalflag', 'ci_count', 'csmtag', 'csm_manager', 'bu_name',
       'internalclient', 'customer_classification',
       'adjustedcustomerclassification', 'manufacturercode',
       'salesorganisationcode', 'documenttype', 'billingblock', 'netvalue',
       'createdbycode', 'ysascurrencycode', 'ysas', 'ysasusd',
       'soldtocustomercode', 'salesofficecode', 'rejec

# Manufacturer Mapping:

In [12]:
contracts["manufacturer_po"] = contracts["manufacturer"]
contracts = contracts.drop(columns=["manufacturer"])

# Re-assigning manufacturer: 
contracts["manufacturer"] = np.where(contracts["portfolio"]=="VBR", contracts["manufacturercode"], contracts["manufacturer_po"])

mask_1 = contracts["portfolio"]=="VBR"
mask_2 = contracts["manufacturer"].isna()
contracts["manufacturer"] = np.where(mask_1 & mask_2, contracts["vendor"], contracts["manufacturer"])

mask_3 = contracts["portfolio"]=="Support Services"
mask_4 = contracts["merged_po"] == 0
contracts["manufacturer"] = np.where(mask_3 & mask_4, contracts["standard_mapping"], contracts["manufacturer"])

mask_5 = contracts["merged_po"] == 1
mask_6 = contracts["manufacturer"].isna()
mask_8 = contracts["manufacturer"] == "DD"
contracts["manufacturer"] = np.where(mask_3 & mask_5 & mask_6 | mask_8, contracts["vendor"], contracts["manufacturer"])

mask_7 = contracts["manufacturer"].isna() 
contracts["manufacturer"] = np.where(mask_3 & mask_5 & mask_7, "DD-UPTIME", contracts["manufacturer"])

contracts["manufacturer"].fillna("DD-UPTIME", inplace= True)

#merged_itsm:
contracts["merged_itsm"]= np.where(contracts["manufacturer"] == contracts["standard_mapping"],1,0)

#Removing manufacturer_po:
contracts=contracts.drop(columns=["manufacturer_po"])

# Adjusting NetValueUSD_x on Final PO and ISM Merged Contracts

In [13]:
# client_country & client_country_manufacturer combinations:
#client_country:
contracts["client_country"] = contracts.adjusted_client_domain_description.astype(str) + "-" + contracts.country.astype(str)

#client_country_manufacturer:
contracts["client_country_manufacturer"] = contracts.client_country.astype(str) + "-" + contracts.manufacturer.astype(str)

# Separate DF for vbr and ss :
mask_1 = contracts["portfolio"]=="VBR"
mask_2 = contracts["portfolio"]=="Support Services"

df_vbr = contracts[mask_1]
df_ss  = contracts[mask_2] 

#Creating TCV split up: only for "Support Services".
total_spend_value = df_ss.groupby("contract_line_number").agg({"spend_value_usd" : "sum","client_country_manufacturer":"nunique"}).reset_index()
total_spend_value.rename(columns = {'spend_value_usd':'total_spend_value','client_country_manufacturer':'count'}, inplace = True)
total_spend_value = pd.DataFrame(total_spend_value)

df_ss = df_ss.merge(total_spend_value, how ="left", on="contract_line_number")
df_ss["net_value_usd_x_1"] = df_ss["netvalueusd_x"]/df_ss["count"]


df_ss["proportion"] = df_ss["spend_value_usd"]/np.where((df_ss["spend_value_usd"]==0)|(df_ss["total_spend_value"]==0),np.nan,df_ss["total_spend_value"])
pos_na = df_ss.index[df_ss["proportion"].apply(np.isnan)]
df_na = df_ss.iloc[pos_na]
df_na["cln"] = df_na["contract_line_number"]
df_na1 = df_na.groupby("contract_line_number").agg({"cln" :"count"}).reset_index()
df_na1.rename(columns = {'cln':'count'}, inplace = True)
df_na1 = pd.DataFrame(df_na1.assign(proportion_new = lambda x: 1/df_na1["count"]))
df_na2 = df_na.merge(df_na1[["contract_line_number","proportion_new"]], how="inner", on="contract_line_number" )
df_ss["proportion"][pos_na] = df_na2["proportion_new"]

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
#Creating TCV split up: only for "VBR".
df_vbr["cln"] = df_vbr["contract_line_number"]
vbr_cli_count = df_vbr.groupby("contract_line_number").agg({"cln" :"count"}).reset_index()
vbr_cli_count.rename(columns = {'cln':'count'}, inplace = True)
vbr_cli_count = pd.DataFrame(vbr_cli_count.assign(proportion = lambda x: 1/vbr_cli_count["count"]))
vbr_cli_count = pd.DataFrame(vbr_cli_count)
df_vbr = df_vbr.merge(vbr_cli_count, how="left", on="contract_line_number")
df_vbr["net_value_usd_x_1"] = df_vbr["netvalueusd_x"]

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# Removing Excess cloumns in df_ss and df_vbr before appending both to one as contracts.
df_vbr=df_vbr.drop(columns=["count"])
df_vbr=df_vbr.drop(columns=["cln"])
df_ss=df_ss.drop(columns=["total_spend_value"])
df_ss=df_ss.drop(columns=["count"])

In [16]:
# Appending df_ss and df_vbr:
contracts_1 = df_ss.append(df_vbr, ignore_index=True)

C:\Users\gramener\Anaconda3\lib\site-packages\pandas\core\frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [17]:
# Net_Value_USD SPLIT :
contracts_1["net_value_usd_x_calc"] = contracts_1["net_value_usd_x_1"] * contracts_1["proportion"]

In [18]:
# Validation Check merged_Netvalue:
merged_netvalue = contracts_1["net_value_usd_x_calc"].sum()

vbr_netvalue = contracts_1[contracts_1['portfolio'].isin(["VBR"])]
vbr_netvalue = vbr_netvalue["net_value_usd_x_calc"].sum()

ss_netvalue = contracts_1[contracts_1['portfolio'].isin(["Support Services"])]
ss_netvalue = ss_netvalue["net_value_usd_x_calc"].sum()

#                             CLIENT_COUNTRY_MANUFACTURER_VIEW

In [19]:
#Removing contract_line_number,contractitemnumber,contractnumber :
contracts_1 = contracts_1.drop(columns=["contract_line_number","contractitemnumber","contractnumber"])

# Creating New Proportions for ccm view

In [20]:
# Separate DF for vbr and ss :
mask_1 = contracts_1["portfolio"]=="VBR"
mask_2 = contracts_1["portfolio"]=="Support Services"

df_vbr = contracts_1[mask_1]
df_ss  = contracts_1[mask_2] 

In [21]:
#Creating TCV split up: only for Support Services.

df_ss["ccm"] = df_ss["client_country_manufacturer"]
total_net_value_ss = df_ss.groupby("client_country_manufacturer").agg({"net_value_usd_x_calc":"sum","ccm":"count"}).reset_index()
total_net_value_ss.rename(columns = {'net_value_usd_x_calc':'total_net_value','ccm':'count'}, inplace = True)
total_net_value_ss = pd.DataFrame(total_net_value_ss)
df_ss = df_ss.merge(total_net_value_ss,how="left",on="client_country_manufacturer")

# since no NA:
df_ss["proportion_1"] = 1/df_ss["count"]

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
# Creating TCV split up: only for VBR.
df_vbr["ccm"] = df_vbr["client_country_manufacturer"]
total_net_value_vbr = df_vbr.groupby("client_country_manufacturer").agg({"net_value_usd_x_calc":"sum","ccm":"count"}).reset_index()
total_net_value_vbr.rename(columns = {'net_value_usd_x_calc':'total_net_value','ccm':'count'}, inplace = True)
total_net_value_vbr = pd.DataFrame(total_net_value_vbr)
df_vbr = df_vbr.merge(total_net_value_vbr,how="left",on="client_country_manufacturer")

# since no NA:
df_vbr["proportion_1"] = 1/df_vbr["count"]


C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
# Appending df_ss and df_vbr:
contracts_1 = df_ss.append(df_vbr, ignore_index=True)

In [24]:
# Net_Value_USD SPLIT :
contracts_1["net_value_usd_x_calc_1"] = contracts_1["total_net_value"] * contracts_1["proportion_1"]

In [25]:
# Validation Check net_value_usd_x_calc_1:
# TCV
merged_netvalue = contracts_1["net_value_usd_x_calc_1"].sum()
merged_netvalue

2770707092.496568

In [26]:
# VBR:
vbr_netvalue = contracts_1[contracts_1['portfolio'].isin(["VBR"])]
vbr_netvalue = vbr_netvalue["net_value_usd_x_calc_1"].sum()
vbr_netvalue

1201418632.4175608

In [27]:
# Support-Services:
ss_netvalue = contracts_1[contracts_1['portfolio'].isin(["Support Services"])]
ss_netvalue = ss_netvalue["net_value_usd_x_calc_1"].sum()
ss_netvalue

1569288460.079007

# ROLL-UP

In [28]:
#Filtering columns:
rolled_up_filter = contracts_1[["client_country", "country","client_country_manufacturer","adjusted_client_domain_description","manufacturer","proportion_1","portfolio","net_value_usd_x_calc_1"]]
rolled_up_data = pd.DataFrame(rolled_up_filter)

In [29]:
#GROUPING- MAIN ROLL-UP:
roll_test = rolled_up_data.groupby(["client_country","country","adjusted_client_domain_description","manufacturer","portfolio"]).agg({"net_value_usd_x_calc_1":"sum"}).reset_index()
roll_test.rename(columns = {'net_value_usd_x_calc_1':'grand_total'}, inplace = True)

#Melt grand_total across VBR and Support Services:
rolled_up_view=roll_test.pivot_table(index=['client_country','country','adjusted_client_domain_description','manufacturer'],columns='portfolio',values='grand_total',fill_value=0).reset_index()
rolled_up_view["grand_total"] = rolled_up_view["Support Services"] +  rolled_up_view["VBR"]

rolled_up_view["client_country_manufacturer"]= rolled_up_view.client_country.astype(str) + "-" + rolled_up_view.manufacturer.astype(str)

#Rearranging:
rolled_up_view = rolled_up_view[["adjusted_client_domain_description","country","client_country","manufacturer","client_country_manufacturer","Support Services","VBR","grand_total"]]



# DERIVED - METRICS

In [30]:
#client_country_manufacturer_both_1_uptime_2_vbr_3:
client_country_manufacturer_both_1_uptime_2_vbr_3 = rolled_up_view.groupby("client_country_manufacturer").agg({"VBR":"sum", "Support Services":"sum"}).reset_index()
client_country_manufacturer_both_1_uptime_2_vbr_3.rename(columns = {'VBR':'total_vbr','Support Services':'total_ss'}, inplace = True)
client_country_manufacturer_both_1_uptime_2_vbr_3 = pd.DataFrame(client_country_manufacturer_both_1_uptime_2_vbr_3)

#Nested condition for assigning 1,2,3,4:
conditions = [(client_country_manufacturer_both_1_uptime_2_vbr_3["total_ss"] != 0) & (client_country_manufacturer_both_1_uptime_2_vbr_3["total_vbr"]!=0),
             (client_country_manufacturer_both_1_uptime_2_vbr_3["total_ss"] != 0) & (client_country_manufacturer_both_1_uptime_2_vbr_3["total_vbr"]==0),
             (client_country_manufacturer_both_1_uptime_2_vbr_3["total_ss"] == 0) & (client_country_manufacturer_both_1_uptime_2_vbr_3["total_vbr"]!=0)]
choices = [1,2,3]
client_country_manufacturer_both_1_uptime_2_vbr_3["client_country_manufacturer_both_1_uptime_2_vbr_3"] = np.select(conditions, choices, default= 4)

#selecting columns for merge:
client_country_manufacturer_both_1_uptime_2_vbr_3 = client_country_manufacturer_both_1_uptime_2_vbr_3[["client_country_manufacturer","client_country_manufacturer_both_1_uptime_2_vbr_3"]]

#merge:
rolled_up_view = rolled_up_view.merge(client_country_manufacturer_both_1_uptime_2_vbr_3, how="left", on= "client_country_manufacturer")
rolled_up_view["client_country_manufacturer_both_1_uptime_2_vbr_3"] = np.where(rolled_up_view["client_country_manufacturer_both_1_uptime_2_vbr_3"].isna(),4,rolled_up_view["client_country_manufacturer_both_1_uptime_2_vbr_3"])


In [31]:
#multiple_overall:
multiple_overall = rolled_up_view.groupby("client_country").agg({"manufacturer":"nunique"}).reset_index()
multiple_overall.rename(columns = {'manufacturer':'count'}, inplace = True)
multiple_overall = multiple_overall[multiple_overall["count"]>1]
multiple_overall = pd.DataFrame(multiple_overall)

multiple_overall["multiple_overall"] = 1

multiple_overall = multiple_overall[["client_country", "multiple_overall"]]

rolled_up_view = rolled_up_view.merge(multiple_overall, how="left", on= "client_country")
rolled_up_view["multiple_overall"] = np.where(rolled_up_view["multiple_overall"].isna(),0,1)
rolled_up_view.multiple_overall.sum()

27659

In [32]:
#client_uptime:
mask_1 = rolled_up_view["Support Services"] != 0

client_uptime = rolled_up_view[mask_1]
client_uptime["c"] = client_uptime["adjusted_client_domain_description"]

client_uptime = client_uptime.groupby("adjusted_client_domain_description").agg({"c":"count"}).reset_index()
client_uptime.rename(columns = {'c':'count'}, inplace = True)
client_uptime = pd.DataFrame(client_uptime)

client_uptime["client_uptime"] = 1
client_uptime = client_uptime[["adjusted_client_domain_description", "client_uptime"]]

rolled_up_view = rolled_up_view.merge(client_uptime, how="left", on= "adjusted_client_domain_description")
rolled_up_view["client_uptime"] = np.where(rolled_up_view["client_uptime"].isna(),0,1)

rolled_up_view.client_uptime.sum()

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


32479

In [33]:
#client_country_uptime:
mask_1 = rolled_up_view["Support Services"] != 0

client_country_uptime = rolled_up_view[mask_1]
client_country_uptime["cc"] = client_country_uptime["client_country"]

client_country_uptime = client_country_uptime.groupby("client_country").agg({"cc":"count"}).reset_index()
client_country_uptime.rename(columns = {'cc':'count'}, inplace = True)
client_country_uptime = pd.DataFrame(client_country_uptime)

client_country_uptime["client_country_uptime"] = 1
client_country_uptime = client_country_uptime[["client_country", "client_country_uptime"]]

rolled_up_view = rolled_up_view.merge(client_country_uptime, how="left", on= "client_country")
rolled_up_view["client_country_uptime"] = np.where(rolled_up_view["client_country_uptime"].isna(),0,1)

rolled_up_view.client_country_uptime.sum()
#343714

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


30398

In [34]:
# Success_play 3 new logic:

#creating client-manufacturer:
rolled_up_view["client_manufacturer"]= rolled_up_view.adjusted_client_domain_description.astype(str) + "-" + rolled_up_view.manufacturer.astype(str)

#selecting data for grouping:
sp3_data = rolled_up_view[["client_manufacturer","VBR","Support Services","grand_total"]]

#grouping by client-manufacturer and getting net value for each client manufacturer group:
sp3_data = sp3_data.groupby("client_manufacturer").agg({"VBR":"sum", "Support Services":"sum"}).reset_index()
sp3_data.rename(columns = {'VBR':'total_vbr','Support Services':'total_ss'}, inplace = True)

# flagging client_manufacturer combination having both vbr and ss:

distinct_client_country = sp3_data[(sp3_data["total_vbr"]!= 0) & (sp3_data["total_ss"]!= 0)]
distinct_client_country["flag"] = 1
distinct_client_country = distinct_client_country[["client_manufacturer","flag"]]

#merging distinct_client_country with rolled_up_view to get flag.
rolled_up_view = rolled_up_view.merge(distinct_client_country, how="left", on="client_manufacturer")
rolled_up_view["flag"] = np.where(rolled_up_view["flag"].isna(),0,1)

mask_1 = rolled_up_view["Support Services"] != 0
mask_2 = rolled_up_view["VBR"] != 0

s3_ss = rolled_up_view[mask_1]
s3_ss = pd.unique(s3_ss["client_country_manufacturer"])
s3_ss = pd.DataFrame(s3_ss)

s3_vbr = rolled_up_view[mask_2]
s3_vbr = pd.unique(s3_vbr["client_country_manufacturer"])
s3_vbr = pd.DataFrame(s3_vbr)

s3_ss["f"] = "ss"
s3_vbr["f"] = "vbr"

s3_ss.rename(columns = {0:'client_country_manufacturer'}, inplace = True)
s3_vbr.rename(columns = {0:'client_country_manufacturer'}, inplace = True)

vbr_left = s3_vbr.merge(s3_ss, how= "left", on="client_country_manufacturer")
vbr_left["check"] = np.where(vbr_left["f_y"].isna(),1,0)
vbr_left = vbr_left[["client_country_manufacturer", "check"]]

rolled_up_view = rolled_up_view.merge(vbr_left, how = "left", on="client_country_manufacturer")
rolled_up_view["check"] = np.where(rolled_up_view["check"].isna(),0,1)
rolled_up_view["check_final"] = np.where((rolled_up_view["flag"]==1) & (rolled_up_view["check"]==1) ,1,0)

C:\Users\gramener\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [35]:
# CALCULATING SUCCESS_PLAYS:

rolled_up_view["success_play_1"] = np.where((rolled_up_view["client_country_manufacturer_both_1_uptime_2_vbr_3"] ==1),1,0)

rolled_up_view["success_play_2"] = np.where(((rolled_up_view["multiple_overall"] ==1) & (rolled_up_view["success_play_1"] ==0) & (rolled_up_view["client_country_uptime"] ==1) & (rolled_up_view["client_country_manufacturer_both_1_uptime_2_vbr_3"] ==3)) ,1,0)

rolled_up_view["success_play_3"] = np.where(((rolled_up_view["client_country_manufacturer_both_1_uptime_2_vbr_3"] ==3) & (rolled_up_view["check_final"]==1)),1,0)

rolled_up_view["success_play_4"] = np.where((rolled_up_view["client_uptime"]==1),1,0)


In [36]:
#Nested condition for success_play:
conditions = [(rolled_up_view["success_play_1"] == 1),
             ((rolled_up_view["success_play_1"]==0) & (rolled_up_view["success_play_2"] ==1)),
             ((rolled_up_view["success_play_1"]==0) & (rolled_up_view["success_play_2"]==0) & (rolled_up_view["success_play_3"]==1)),
             ((rolled_up_view["success_play_1"]==0) & (rolled_up_view["success_play_2"]==0) & (rolled_up_view["success_play_3"]==0) &(rolled_up_view["success_play_4"]==1))]
choices = [1,2,3,4]
rolled_up_view["success_play"] = np.select(conditions, choices, default="No Success Play")


# Validating Success_Play Netval:

In [37]:
# SS_1
SS_1_netval = rolled_up_view[rolled_up_view["success_play_1"]==1]
SS_1_netval = SS_1_netval["grand_total"].sum()
SS_1_netval

650547897.4114052

In [38]:
# SS_2
SS_2_netval = rolled_up_view[rolled_up_view["success_play_2"]==1]
SS_2_netval = SS_2_netval["grand_total"].sum()
SS_2_netval

395792054.95038074

In [39]:
# SS_3
SS_3_netval = rolled_up_view[rolled_up_view["success_play_3"]==1]
SS_3_netval = SS_3_netval["grand_total"].sum()
SS_3_netval

246102090.66820613

In [40]:
# SS_4
SS_4_netval = rolled_up_view[rolled_up_view["success_play_4"]==1]
SS_4_netval = SS_4_netval["grand_total"].sum()
SS_4_netval

2480304172.736837

In [41]:
# Creating TPIC_FLAG:client_country:
Tpic = rolled_up_view.groupby("client_country").agg({"manufacturer":"nunique"}).reset_index()
Tpic.rename(columns = {'manufacturer':'count'}, inplace = True)
Tpic = Tpic[Tpic["count"]>1]
Tpic["TPIC_FLAG"] = 1
Tpic = Tpic[["client_country","TPIC_FLAG"]]
Tpic = pd.DataFrame(Tpic)

rolled_up_view= rolled_up_view.merge(Tpic, how="left" , on="client_country")
rolled_up_view["TPIC_FLAG"] = np.where((rolled_up_view["TPIC_FLAG"].isna()),0,1)

In [42]:
# rearranging:
rolled_up_view_1 = rolled_up_view[["client_country","country","adjusted_client_domain_description","manufacturer","Support Services","VBR","grand_total","success_play_1","success_play_2","success_play_3","success_play_4","success_play","TPIC_FLAG"]]

In [43]:
rolled_up_view_1["grand_total"].sum()


2735034362.55196

In [142]:
#Output:
df1 = pd.DataFrame(rolled_up_view_1)
df1.to_excel("CCM_view_python_op.xlsx")